In [ ]:
import pandas as pd
import numpy as np

import readability
import syntok.segmenter as segmenter

dataset = 'training'
# dataset = 'valid'
# dataset = 'test'
CSV_URL = f'../data/{dataset}_set.csv'

In [ ]:
df = pd.read_csv(CSV_URL)
df.drop_duplicates(inplace=True)
df = df.sample(frac=1, random_state=9).reset_index(drop=True)
df_reduced = pd.DataFrame(df[['ID', 'Sentence DE']])

In [ ]:
def get_features(text):
    tokenized = '\n\n'.join(
        '\n'.join(' '.join(token.value for token in sentence)
                  for sentence in paragraph)
        for paragraph in segmenter.analyze(text))
    return readability.getmeasures(tokenized, lang='de', merge=True)


features = df['Sentence DE'].apply(get_features)

In [ ]:
feature_names = []
for idx, row in features.items():
    for k, v in row.items():
        if k not in feature_names:
            feature_names.append(k)
features_dict = {name: np.full(len(df), np.nan) for name in feature_names}
for idx, row in features.items():
    for k, v in row.items():
        features_dict[k][idx] = v

df_features = pd.DataFrame.from_dict(features_dict)
df_features = df_reduced.join(df_features)

In [ ]:
df_features.to_csv(f'../data/features/features_{dataset}_readability_de.csv', index=False)